In [91]:
from gcp_interaction import read_blob_to_pandas, write_to_blob_csv
import json
import pandas as pd
import numpy as np

In [92]:
with open("excess-energy-prediction-393ec78547e4.json", "r") as f:
    gcp_login_info = json.load(f)
with open("gcp_info.json", "r") as f:
    bucket_name = json.load(f)["bucket_name"]

# Actual Demand

In [93]:
# Example URL 'Raw_Data/EirGrid/ALL/demandActual/2017/demandActual_ALL_2017_Apr.csv'
base_url = r"Raw_Data/EirGrid/ALL/demandActual"
years = ['2017', '2018', '2019', '2020']
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [94]:
dfs = []
for year in years:
    for month in months:
        url = base_url+"/"+year+"/"+"demandActual"+"_ALL"+"_"+year+"_"+month+".csv"
        dfs.append(read_blob_to_pandas(bucket_name, url))

In [95]:
demand_df = pd.concat(dfs)
demand_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140256 entries, 0 to 2975
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Unnamed: 0            140256 non-null  int64 
 1   DATE & TIME           140256 non-null  object
 2    ACTUAL DEMAND(MW)    140256 non-null  object
 3    FORECAST DEMAND(MW)  140256 non-null  object
 4    REGION               140256 non-null  object
dtypes: int64(1), object(4)
memory usage: 6.4+ MB


In [96]:
demand_df.isna().sum().sum()

0

In [97]:
demand_df.nunique()

Unnamed: 0                2976
DATE & TIME             140256
 ACTUAL DEMAND(MW)        7501
 FORECAST DEMAND(MW)         1
 REGION                      1
dtype: int64

No info for forecast demand so I'm going to drop it. 

In [98]:
demand_df = demand_df.drop(['Unnamed: 0', ' FORECAST DEMAND(MW)'], axis = 1, errors = 'ignore')
demand_df["DATE & TIME"] = pd.to_datetime(demand_df["DATE & TIME"], infer_datetime_format=True, errors="ignore")
demand_df = demand_df.rename(columns=lambda x: x.strip())

In [99]:
demand_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140256 entries, 0 to 2975
Data columns (total 3 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   DATE & TIME        140256 non-null  datetime64[ns]
 1   ACTUAL DEMAND(MW)  140256 non-null  object        
 2   REGION             140256 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 4.3+ MB


In [100]:
demand_df.head()

,DATE & TIME,ACTUAL DEMAND(MW),REGION
0,2017-01-01 00:00:00,3696,All Island
1,2017-01-01 00:15:00,3661,All Island
2,2017-01-01 00:30:00,3622,All Island
3,2017-01-01 00:45:00,3492,All Island
4,2017-01-01 01:00:00,3457,All Island


In [101]:
write_to_blob_csv(
    bucket_name=bucket_name, df=demand_df, filepath="transformed/demand.csv")

transformed/demand.csv uploaded to <Bucket: excess-energy-raw-data>


# Actual Generation

In [102]:
# Example URL 'Raw_Data/EirGrid/ALL/demandActual/2017/demandActual_ALL_2017_Apr.csv'
base_url = r"Raw_Data/EirGrid/ALL/generationActual"
years = ['2017', '2018', '2019', '2020']
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [103]:
dfs = []
for year in years:
    for month in months:
        url = base_url+"/"+year+"/"+"generationActual"+"_ALL"+"_"+year+"_"+month+".csv"
        dfs.append(read_blob_to_pandas(bucket_name, url))

In [104]:
generation_df = pd.concat(dfs)
generation_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140256 entries, 0 to 2975
Data columns (total 4 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Unnamed: 0              140256 non-null  int64  
 1   DATE & TIME             140256 non-null  object 
 2    ACTUAL GENERATION(MW)  140190 non-null  float64
 3    REGION                 140256 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 5.4+ MB


## Filling NAs

In [105]:
generation_df.isna().sum()

Unnamed: 0                 0
DATE & TIME                0
 ACTUAL GENERATION(MW)    66
 REGION                    0
dtype: int64

In [106]:
generation_df[generation_df[' ACTUAL GENERATION(MW)'].isna()]

,Unnamed: 0,DATE & TIME,ACTUAL GENERATION(MW),REGION
2404,2404,26 March 2017 01:00,NaN,All Island
2405,2405,26 March 2017 01:15,NaN,All Island
2406,2406,26 March 2017 01:30,NaN,All Island
2407,2407,26 March 2017 01:45,NaN,All Island
2116,2116,23 June 2017 01:00,NaN,All Island
...,...,...,...,...
2692,2692,29 March 2020 01:00,NaN,All Island
2693,2693,29 March 2020 01:15,NaN,All Island
2694,2694,29 March 2020 01:30,NaN,All Island
2695,2695,29 March 2020 01:45,NaN,All Island


It would be good to fill the null values but that falls apart if there are a lot in a row. We can use a function to see the longest stretch of NAs

In [107]:
def max_na(s):
    isna = s.isna()
    blocks = (~isna).cumsum()
    return isna.groupby(blocks).sum().max()

max_na(generation_df[' ACTUAL GENERATION(MW)'])

46

In [108]:
len(generation_df)

140256

46 NA's in a row isn't great but out of 140,256 total rows I think it's okay to fill. 

In [109]:
generation_df[' ACTUAL GENERATION(MW)'] = generation_df[' ACTUAL GENERATION(MW)'].fillna(method = 'backfill')

In [110]:
generation_df.isna().sum()

Unnamed: 0                0
DATE & TIME               0
 ACTUAL GENERATION(MW)    0
 REGION                   0
dtype: int64

In [111]:
generation_df = generation_df.drop('Unnamed: 0', axis = 1, errors = 'ignore')
generation_df = generation_df.replace({"-":np.nan})
generation_df["DATE & TIME"] = pd.to_datetime(generation_df["DATE & TIME"], 
                                              infer_datetime_format=True, 
                                              errors="ignore")
generation_df = generation_df.rename(columns=lambda x: x.strip())

In [180]:
generation_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140256 entries, 0 to 2975
Data columns (total 3 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   DATE & TIME            140256 non-null  datetime64[ns]
 1   ACTUAL GENERATION(MW)  140256 non-null  float64       
 2   REGION                 140256 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 8.3+ MB


In [112]:
write_to_blob_csv(
    bucket_name=bucket_name, df=generation_df, filepath="transformed/generation.csv")

transformed/generation.csv uploaded to <Bucket: excess-energy-raw-data>


# Wind

In [113]:
# Example URL 'Raw_Data/EirGrid/ALL/demandActual/2017/demandActual_ALL_2017_Apr.csv'
base_url = r"Raw_Data/EirGrid/ALL/windActual"
years = ['2017', '2018', '2019', '2020']
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [114]:
dfs = []
for year in years:
    for month in months:
        url = base_url+"/"+year+"/"+"windActual"+"_ALL"+"_"+year+"_"+month+".csv"
        dfs.append(read_blob_to_pandas(bucket_name, url))

In [143]:
wind_df = pd.concat(dfs)
wind_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140272 entries, 0 to 2975
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Unnamed: 0          140272 non-null  int64 
 1   DATE & TIME         140272 non-null  object
 2    FORECAST WIND(MW)  140272 non-null  object
 3     ACTUAL WIND(MW)   140272 non-null  object
 4    REGION             140272 non-null  object
dtypes: int64(1), object(4)
memory usage: 6.4+ MB


In [144]:
wind_df.isna().sum()

Unnamed: 0            0
DATE & TIME           0
 FORECAST WIND(MW)    0
  ACTUAL WIND(MW)     0
 REGION               0
dtype: int64

In [145]:
wind_df = wind_df.drop('Unnamed: 0', axis = 1, errors = 'ignore')
wind_df = wind_df.replace({"-":np.nan})
wind_df["DATE & TIME"] = pd.to_datetime(wind_df["DATE & TIME"], 
                                              infer_datetime_format=True, 
                                              errors="ignore")
wind_df = wind_df.rename(columns=lambda x: x.strip())

In [146]:
wind_df.head()

,DATE & TIME,FORECAST WIND(MW),ACTUAL WIND(MW),REGION
0,2017-01-01 00:00:00,1172,1214,All Island
1,2017-01-01 00:15:00,1198,1168,All Island
2,2017-01-01 00:30:00,1224,1131,All Island
3,2017-01-01 00:45:00,1250,1025,All Island
4,2017-01-01 01:00:00,1280,1123,All Island


In [127]:
write_to_blob_csv(
    bucket_name=bucket_name, df=wind_df, filepath="transformed/wind.csv")

transformed/wind.csv uploaded to <Bucket: excess-energy-raw-data>


# Combining

In [147]:
print(f'Demand length: {len(demand_df)}')
print(f'Generation length: {len(generation_df)}')
print(f'Wind length: {len(wind_df)}')

Demand length: 140256
Generation length: 140256
Wind length: 140272


In [160]:
wind_dates = set(wind_df['DATE & TIME'])
len(wind_dates)

140256

Interestingly there must be some repeated dates in the wind dataset. 

In [169]:
wind_df[wind_df.duplicated(subset = 'DATE & TIME', keep = False)].head(6)

,DATE & TIME,FORECAST WIND(MW),ACTUAL WIND(MW),REGION
2692,2017-10-29 01:00:00,1510,1882,All Island
2693,2017-10-29 01:00:00,1450,1882,All Island
2694,2017-10-29 01:15:00,1493,1932,All Island
2695,2017-10-29 01:15:00,1449,1932,All Island
2696,2017-10-29 01:30:00,1476,1935,All Island
2697,2017-10-29 01:30:00,1447,1935,All Island


Going to drop these duplicates which should make all the dataframes the same length.

In [183]:
wind_df = wind_df.drop_duplicates(subset = 'DATE & TIME', keep = 'first')
wind_df = wind_df.reset_index(drop=True)

In [184]:
print(f'Demand length: {len(demand_df)}')
print(f'Generation length: {len(generation_df)}')
print(f'Wind length: {len(wind_df)}')

Demand length: 140256
Generation length: 140256
Wind length: 140256


In [193]:
wind_df = wind_df.reset_index(drop=True)
demand_df = demand_df.reset_index(drop=True)
generation_df = generation_df.reset_index(drop=True)
# Dropping region from two dfs to avoid duplicate cols when merging
generation_df = generation_df.drop(['REGION','DATE & TIME'], axis = 1, errors = 'ignore')
demand_df = demand_df.drop(['REGION','DATE & TIME'], axis = 1, errors = 'ignore')

In [196]:
merged_df = pd.concat([demand_df, wind_df, generation_df], axis = 1)
merged_df.head()

,ACTUAL DEMAND(MW),DATE & TIME,FORECAST WIND(MW),ACTUAL WIND(MW),REGION,ACTUAL GENERATION(MW)
0,3696,2017-01-01 00:00:00,1172,1214,All Island,4126.0
1,3661,2017-01-01 00:15:00,1198,1168,All Island,4184.0
2,3622,2017-01-01 00:30:00,1224,1131,All Island,4187.0
3,3492,2017-01-01 00:45:00,1250,1025,All Island,4059.0
4,3457,2017-01-01 01:00:00,1280,1123,All Island,4094.0


In [198]:
reorder_cols = ['DATE & TIME','REGION',  
                'FORECAST WIND(MW)','ACTUAL WIND(MW)', 
                'ACTUAL DEMAND(MW)','ACTUAL GENERATION(MW)']
merged_df = merged_df[reorder_cols]

In [200]:
write_to_blob_csv(
    bucket_name=bucket_name, df=merged_df, filepath="transformed/EIR_grid_merged.csv")

transformed/EIR_grid_merged.csv uploaded to <Bucket: excess-energy-raw-data>
